# **Amazon Video Games Ratings**
# **Reviews from Amazon customers from October 14, 1999 to July 22, 2014.**

## **Classification of reviews as positive or negative - ranking of 10 top and 10 worst products**

Dataset from Huggin Face, **"LoganKells/amazon_product_reviews_video_games"** provides information about customer reviews and ratings for video games sold by Amazon. I intend to use this data to extract both the 10 best and the 10 worst games, as they have been qualified by the customer-reviewers. Since the dataset is not labeled, I will classify the reviews as positive or negative.
To shorten the elimination process, I will only work with games that have 100 or more reviews for the stated period..

**Features of the dataset:**  
  
1) Unnamed: 0: Internal index column from the source CSV.  
2) reviewerID: Unique identifier for the reviewer.  
3) asin: Amazon Standard Identification Number for the game being reviewed.  
4) reviewerName: Reviewer's display name.  
5) helpful: A pair of integers representing helpful votes received and total votes.  
6) reviewText: Full text of the review.  
7) overall: Rating given by the reviewer.  
8) summary: Short summary or headline of the review.  
9) unixReviewTime: Timestamp of the review in Unix epoch seconds.  
10) reviewTime: Human-readable date of the review.



## **1. Installations and Imports**

In [ ]:
# Installing ub for smooth packages inatallation

!pip install uv -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 63.0 MB/s eta 0:00:00


In [ ]:
# Installing uv to facilitate other packages installations

!uv init

Initialized project `content`


In [ ]:
# Installing packages for LLM, token count, session information

!uv pip install -q openai==1.55.3 tiktoken==0.6.0 session-info --quiet

In [ ]:
# Importing libraries for numerical and data operations and manipulation.
# Importing textsplitter by langchain for chunking

import numpy as np
import pandas as pd
import warnings

from langchain.text_splitter import RecursiveCharacterTextSplitter #for chunking
warnings.filterwarnings('ignore')


In [ ]:
# Importing all Python packages required to access the Azure Open AI API.
# Importing additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

from collections import Counter
from tqdm import tqdm

In [ ]:
session_info.show()

## **2. API authentication**

In [ ]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [ ]:
creds = json.loads(data)

In [ ]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [ ]:
deployment_name = creds["CHATGPT_MODEL"]

In [ ]:
deployment_name

'gpt-4o-mini'

## **3. Utilities**

In [ ]:
# To count tokens for prompt technique perfornance and cost

def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

## **4. Preparing the data**

In [ ]:
df = pd.read_csv('data.csv', on_bad_lines='skip')

In [ ]:
df.head(3)

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,0.0,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,3.0,Good rally game,1372550400,"06 30, 2013"
2,2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,0.0,Wrong key,1403913600,"06 28, 2014"


In [ ]:
df.shape

(50000, 10)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      50000 non-null  int64  
 1   reviewerID      50000 non-null  object 
 2   asin            50000 non-null  object 
 3   reviewerName    49872 non-null  object 
 4   helpful         50000 non-null  object 
 5   reviewText      49998 non-null  object 
 6   overall         50000 non-null  float64
 7   summary         49990 non-null  object 
 8   unixReviewTime  50000 non-null  int64  
 9   reviewTime      50000 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 3.8+ MB


In [ ]:
from datetime import datetime

# Convert timestamps to datetime objects
dates = [datetime.utcfromtimestamp(ts) for ts in df['unixReviewTime']]

# Get the first and last review dates
start_date = min(dates)
end_date = max(dates)

# Calculate number of days covered
days_covered = (end_date - start_date).days
years = round(days_covered/ 365.2425, 2)

print("First review date:", start_date)
print("Last review date:", end_date)
print("Number of days covered:", days_covered)
print(f"Number of years: {years}")

First review date: 1999-10-14 00:00:00
Last review date: 2014-07-22 00:00:00
Number of days covered: 5395
Number of years: 14.77


## **Observations**

- Dataset contains 50,000 rows ans 10 variables.
- Data was collected for 14.77 years.
- There are 7 columns of data type object, 2 integers, and 1 float.
- Size of the dataset is 3.8+ MB.
- For the purpose of this project, features of interest are:  
a) asin  
b) reviewText  
c) overall  

There are several columns that are not useful for our proposed objective. For this reason and to reduce the size of the dataset, we are eliminating useless features. We will keep the product code, reviews, and overall rating.

In [ ]:
# to reduce the size of the dataset to the useful variables

data= df[['asin', 'reviewText', 'overall']]

In [ ]:
# to view the complete review per row

pd.set_option('display.max_colwidth', None)

In [ ]:
# let's check data

data.sample(2)

,asin,reviewText,overall
29521,B000070IWB,This game looks weird without Mario. Besides your doing what wario allways does. Stealing treasure. I bet this will be hard.,0.0
7003,B000035XR9,"This is one of those consoles where you could feel a change in gaming, like the Atari or NES before it. The games were great, mostly side scroll like anything else back then. The games made by Sega were the best. You should get one if you've never played it before, or even if you have.",4.0


In [ ]:
# Number of different video games in the data.

data['asin'].nunique()

3400

There are 3,400 different video games in the dataset.
I will only pick video games that have a minimum of 100 reviews. This comes up to almost seven reviews per year.

In [ ]:
# Eliminating products that have less than 100 reviews. Resetting the index.

data_100min = data.groupby('asin').filter(lambda x: len(x) >= 100).reset_index(drop=True)

In [ ]:
data_100min.head(2)

,asin,reviewText,overall
0,B00000DMB3,"DO not get me wrong. I am one of The BIGGEST Zelda fans in the world. and I bought this one as soon as it came out with the gold cartige and all.But this Just isn't a Zelda game....Its missing the Fancy tunes, the story just doesn't seem that addictive.WHen you first get the game its great and it looks to be one of the best games. but then after you beat it.... its kinda boring...Zelda games should never get boring! Zelda NES I STILL paly that damn game.' Link Not as good as the first but still I like playing it. Zelda A Link To the pass on SNES is the BEST Zelda game.I just cannot love this game.. it did horrorble for a Zelda game.",2.0
1,B00000DMB3,"Ocarina of Time is the best video game of the last 100 years.If you have never played it, your missing out. Plus, you are a tool.",4.0


In [ ]:
data_100min.sample(3)

,asin,reviewText,overall
3691,B00006IJJK,"One of the best games ever made. Beautiful environments, great control, deep story. This game is nearly flawless.",4.0
3999,B00008J7NZ,"Halo 2 is a good game, but not perfect. The campaign is fun, a little short, but fun. The levels are great on any difficulty, and I love the Elite missions. They are fun and challenging, a great twist to the game. The only two things I wished that did not happen in the game was the short storyline, and the ending. They leave you hanging, waiting for the third Halo. Campaign is fun, but it is not a fun game unless it has a good multiplayer. The multiplayer maps are great, but there are not enough of them. Bungie could have waited a while to bring out the game to make more multiplayer maps. Overall a good game.Campaign- 4 starsMultiplayer- 4 starsFun- 5 starsAlso here are some cheats:To get the blind skull start the map Outskirts. Walk through the first door and crouch jump onto one of the lights. Then look to the leftand you will see a wall. Run at the wall and crouch jump onto that wall. Walk foward about 10 steps and look left. You should see a hallway. Go to the end of that hallway and press x to pick up the skull.",4.0
1329,B00005NZ1G,All I can say is that this game was made for the Xbox. I had so much fun with it. The game has great graphics and is vey intence. If you don't have Halo already then I suggest that you buy it. You will probably enjoy it as much as I have.,4.0


In [ ]:
# Let's check our new dataset size

data_100min.shape

(5674, 3)

Out of 50,000 product reviews, only 5,674 products have been reviewed and rated more than 100 times in the indicated period.

In [ ]:
# let's check for missing values

data_100min.isnull().sum()

,0
asin,0
reviewText,0
overall,0


There are no missing values in the data.



## **5. Prompt Engineering**

Let's classify these reviews as positive or negative with the use of an LLM to perform the classification. In this case it will be gpt-4o-mini.

I will concatenate all these reviews into one string to pass it on the LLM to provide me with the classification

In [ ]:
all_reviews_string = data_100min['reviewText'].str.cat(sep='||')

In [ ]:
all_reviews_string

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
text = all_reviews_string

In [ ]:
# Counting the tokens in the string all_review_string

def count_tokens_in_string(text, model="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [ ]:
count_tokens_in_string(text, model="gpt-3.5-turbo")

983565

In [ ]:
token_limit = 8192

The tokens for the concatenated string reach 983,565. LLM resource has a low tier of 100K token limit. To preserve semantic coherence, and make sure text fits the token limits, I cannot "chunk" this string, because the model will be confused as it will be reviewing overlap text as well possibly more than one review. I have to instruct the model to follow the separator || I included when creating variable 'all reviews_string'.

Let's craft the zero_shot prompt to send to the LLM and classify the reviews

In [ ]:
# The system message is as follows:

zero_shot_system_message = """
    You are a sentiment classification expert. Your task is to read an entire product review
    carefully — from beginning to end — and then classify the overall sentiment as either
    'positive' or 'negative'. Even if the review contains both praise and criticism, choose
    the sentiment that reflects the reviewer’s final judgment or overall tone. Do not
    misunderstand user's recommendations with a final judgement. Respond with only
    one word: 'positive' or 'negative'. Do not explain.
"""


In [ ]:
# Creating a user_message_template

user_message_template = """```reviewText```"""

In [ ]:
for review in df['reviewText']:
    zero_shot_prompt = [
        {'role': 'system', 'content': zero_shot_system_message},
        {'role': 'user', 'content': user_message_template.format(reviewText=review)}
    ]


Let's test our prompt with 5 randomly selected reviews

In [ ]:
import random
import textwrap

# Randomly select 5 indices from the reviewText column
sample_indices = random.sample(range(len(df)), 5)

for count, idx in enumerate(sample_indices, start=1):
    review_text = df['reviewText'].iloc[idx]

    # Send the prompt to the model
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=zero_shot_prompt,
            temperature=0,
            max_tokens=4
        )

        # Extract sentiment
        sentiment = response.choices[0].message.content.strip()

        # Print review and sentiment
        print(f"\n🆔 Review Index: {idx}")
        print("📝 Review:\n", textwrap.fill(review_text, width=140))
        print("🔎 Predicted Sentiment:", sentiment)
        print("-" * 60)

    except Exception as e:
        print(f"\n❌ Error on review #{count} (Index: {idx}): {e}")

## **Observations and actions**

**1.-** I started this project with a deployed model gpt-4o-mini. But it was not delivering the right classification. It is a good model to optimize cost but not for nuanced understanding. I will move up to gpt-4o that offers better reasoning and language comprehension, especially for sentiment in ambiguous or mixed reviews. I am expecting better results.  


**2.-** I ran it with gpt-4o and a revised prompt. I get better results (not mostly negative classifications). But, still, misclassifications are too common to consider this a good project. I think the widespread concept that "for sentiment analysis, zero_shot prompt is the way to go" does not apply always if you are pursuing a thorough, good classifier model. Maybe. that idea works for already classified data that just needs to have good predictive capabilities.  


**3.-** To improve it to a satisfactory level, I will use few_shot prompt engineering. I will do this with gpt-4o-mini. If the outpcome is not satisfactory, I will try with gpt-4o.  

**4.-** Below are a sample of 19 classifications done with zero_shot prompting. Most of them are misclassifications. I evaluated them and classify them. The Predicted Sentiment output was generated with zero_shot sampling. My evaluation (to the right of Predicted Sentiment), which I'm naming Correct Sentiment, are the Groud Truths. I will employ these Ground Truths as Gold Examples for the few_shot prompting.

## **6. Evaluation and Ground Truth**

🆔 Review Index: 35111 - Count: 1  
 📝 Review:  
I have been a huge fan of Harvest Moon since I first played it on my gameboy years ago. Harvest Moon: Back to Nature for PS One has been my
favorite, and I'm so glad to know that A Wonderful Life will be bringing back the marriage aspect which I know a lot of people missed with
Save The Homeland.  Harvest Moon: A Wonderful Life will bring back my favorite qualities (ie dating, marriage, time progression) with some
new features as well. I've heard there will be new farm animals (pigs?) and the ability to cross vegetables (wow!). The animation is as cute
as ever, and I can't wait to hear the music. I will be very excited to receive my copy of Harvest Moon: A Wonderful Life.  

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE

🆔 Review Index: 44710 - Count 2  
📝 Review:  
The Game is not even playable single player its online only with 0 Servers .Why do they sell Broken games Wasted time & money for a game i
can ever use sad  

🔎 Predicted Sentiment: positive - Correct Sentiment: POSITIVE

🆔 Review Index: 45797 - Count 3  
📝 Review:  
Veery addictive good for those long waits in very boring places (doctors offices and car trips) needs online capability

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE

🆔 Review Index: 35999 - Count 4  
📝 Review:  
 Boktai's plot is integrated around its one key gimmick: as Django, you hunt vampires, and you need sunshine (that's right! Real sunshine!)
to do it.  Game environments even alter themselves depending on whether it is day or night.  Perfect.  But other aspects of gameplay are
remarkable, as well: in keeping with the Metal Gear design, you are urged to sneak around and strategize.  In fact, limited sun-power during
nighttime gameplay actually necessitates it.It's true that this a new way of playing.  Still, sometimes the gameplay and puzzles are a
little dull and repetitive--which is heartbreakingly disappointing given the attention paid to all other aspects of game design.  Boktai is
conceptually thrilling, but the actual game is just sort of... not.  Gameplay simply isn't quite what it should have been.Still, sheer
innovation tops Boktai's design flaws.  And absolutely nothing is more exciting than going to the park on a sunny day, finding a bench in a
nice patch of light, turning on my GBA, and hearing it say to me, &quot;Today is one great Boktai day!&quot; I can't help but think to
myself, &quot;Isn't it?&quot;  

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE

🆔 Review Index: 15090 - Count 5  
📝 Review:  
 The superboy plays Gameboy games on your snes (it works on retro duo too!). There is a border for the game that looks like a gameboy. You
can also choose borders and color your own, paint and doodle.  For some games (ex: Donkey Kong (arcade) there are special themes borders
and...it's in color!  Your  games will sound awesome when played on your t.v., those bleeps and bloops will really shine! You can also
change the color pattern of games to your liking.  10/10 highly recommended, buy it!  

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE

🆔 Review Index: 40737 = Count 6  
📝 Review:  
I don't normally play video games, but I'm a huge fan of Civilization 3.  oo huge in fact - it's a real problem for me.  I find myself
skipping meals when I play this game.  Civilization 3 is my favorite of the Civilization series.  I think the zone of controls concept is
natural to the gameplay, and the ability to have cultural takeovers of other cities is fun.  Some of the oddities and cheapness of the
previous versions have been removed - it's not really possible to build every single wonder, for instance.I hadn't played the game in a
while, so I decided to get this complete package when I saw it at the store.  However, the several years of updates have really hurt this
game.  The problems haven't been addressed - most notably, the diplomacy AI is terrible, and the endgame, with its emphasis on large amounts
of indivisual military units, is so ridiculously slow that I can still only rarely make it through without quitting, frustrated.  Instead,
the main change is that resources are much more scarce.  These recourses are essential to the progress of your game.  However you have no
way of knowing if you'll have, say, a coal resource somewhere in your map, until you've already been playing for 5 hours.If the scarcity was
a matter of instigating small military attacks on your neighbors, or heavy diplomacy, that would be fine.  However, this weakness plays off
the game's other weakenesses.  Often the necessary resource will only be available halfway around the world.  Launching a military attack is
so ridiculously slow that it isn't worth the time even for the most addictive player (that being: me).  And the diplomacy AI is so worthless
that other empires won't trade their essential resources even when you throw ridiculous sums of money and technology at them.Anyway, it's no
fun to play for hours and hours before you find that your game is a lost cause due to the map you were given.  Also, the expansion disks are
(to my mind) worthless.  So I'd strongly recommend just buying the earlier editions of Civilization 3, which are much cheaper.  

🔎 Predicted Sentiment: positive - Correct Sentiment: NEGATIVE

🆔 Review Index: 34045 - Count 7  
📝 Review:  
 1080 Avalanche is definately a fun game in the sense of speed, but it comes to an overall game, such as tricks and custimization, buy
SSX3!Control: 3/5Im sorry, but I have to have big air and tricks. The way I see it, if I can't pull of awesome combos, things are bad. But,
the system where you can get back on balance if you mess up a jump is cool.Sound: 5/5The reason Sound gets 5? I LOVE THE SOUNDTRACK. The
song &quot;Choke&quot; is amazing, and is really something that you could snowboard to. The sound of the snow under the board is crisp as
well.Modes and Extras: 2/5The Match Race is pretty much the only thing worth it, and even that isn't very good. The characters are limited,
the boards are limited, and there isn't really any customization or unlockables.Overall: 3/5The game is cool if you like speed, but the
tricks suck and you can only hold a jump for 2 seconds. I reccomend you buy SSX3 instead of this, or you can buy this too. Whatever floats
your boat!  

🔎 Predicted Sentiment: positive - Correct Sentiment: NEGATIVE

🆔 Review Index: 12360 - Count 8  
📝 Review:  
The story was OK, I was not taken away with it since it's the usual "Last of species, princess in trouble, hero with daddy-problem"
stuff.The characters had some personality, it was never developed beyond looking like comic characters, I just never took them seriously.The
graphics are ugly, I do not like that they are so small, I could barely see Vivi.But the absolute worst part is the battle-system, it almost
takes a whole minute just to load all the characters, monsters and angle the camera correctly.So much time is wasted just wating for the
battle to load.So when you meet weak monsters you kill with a blow, you seriously get more than annoyed because it takes longer time to load
the fight than to actually finish it!Sad way to ruin a possibly could be fantastic game.  

🔎 Predicted Sentiment: positive - Correct Sentiment: NEGATIVE

🆔 Review Index: 39614 - Count 9  
📝 Review:  
When you add the two together you should come up with one heck of a game.  What happened here is that the strategic, political and conscript
aspects from ROTK were summarized into pairs, and issued to you as suggestions by your leaders.  You can refuse their ideas, and save your
gold, which you need to hire captured enemy generals.  The map is a bit smaller than ROTK and with far less provinces.  Despite this, the
game is incredible.  If you have ever wanted to fight a la DW while you were playing ROTK, this is it...  You make your plans, hire men,
whatever, then choose to attack or perhaps you sit and watch this turn unfold, saving money and manpower.  Unlike DW though, the gates are
everywhere, and they are now strongholds, which you must take to gain the edge.  Also, generals can die 3 times in one battle.  Oh yeah,
they come back 3 times, and get harder each time.  In a normal battle you might have to kill 15 generals... Actually only 5 but you did it 3
times each.  They re-appear out of nowhere with a portion of their original force.  Kind of annoying.  A cool aspect of this is... For once
you can finally be the only general on the screen, if you want that is.  Send yourself in alone against 50000 enemy troops.  The weapons and
items are the same but harder to come by.  Your provinces give you certain items, that you can only use once every 5 turns.  Be carefull
what you use.  Your weapon gets stronger based on your level, just like DW.  The number of men under your command is based on your
achievements like in ROTK.  Form alliances and help defend other lords.  You gain rewards for this.  During battle your AI generals can use
their "skills" once per battle like in ROTK.  Create an officer is better but nothing to get excited about.  You can choose from a few
different looks but the bottom line is you still look stupid.  Oh, and I have yet to see any of the trademark Bodyguards.  I miss that.
Basically, Koei has taken good parts of each game and put it together.  What more could you want?  Oh yeah, no two player co-op, so I guess
that is one thing you could want.  

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE

🆔 Review Index: 2018 - Count 10  
📝 Review:  
This game could have been a heck of alot better. The idea of the game is pretty stupid. They should have taken things from different
episodes and made that the game. The graphics are good, since it's only playstation. The multiplayer is pretty fun. Too bad this game wasn't
released later on and they could have put Jimmy and Timmy in it. I'm glad I bought this game though.  

🔎 Predicted Sentiment: positive - Correct Sentiment: POSITIVE

🆔 Review Index: 26924 - Count 11  
📝 Review:  
This is a great game. As another reviewer said, they could have sold a million copies of a mediocre game, but this one is very good.My seven
year old daughter and I played "Socerer's Stone" first. We enjoyed it, but it was a bit clunky. Next we started playing this one. We're
almost 20 hours into it already, less than half way through, and having a great time. This one has great replay value-we're going to go back
and find the wizard cards we missed.This one took everything that was good about the first game, got rid of what didn't work, and added a
lot more. So you get more spells, more places to explore, more options, more fun.If you get this in the "World of Harry Potter" collection,
you'll get four games for twenty smackers. Great value!  

🔎 Predicted Sentiment: positive -Correct Sentiment: POSITIVE

🆔 Review Index: 30830 - Count 12  
📝 Review:  
 This is a game worth playing and since it's been out about 2 years or a little longer it's affordable. I'm a fan of the original Wolfenstein
3D and this game is still worthy of the name. Great game play, sound, weapons and decent graphics. Another plus about this is when you beat
the game you can play the original Wolfenstein 3D game, this is definally worth buying. Once you get used to the controls and how the game
is played out you'll get hooked.  

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE


🆔 Review Index: 17831 - Count 13  
📝 Review:  
good, but games like golden sun are a lot better. heres how it lays out:graphics:2sound:3 nothing specialstory:4gameplay:3 too much
magicoverall:3.5 if u want an Rpg, get golden sun instead.  

🔎 Predicted Sentiment: positive - Correct Sentiment: POSITIVE

🆔 Review Index: 24291 - Count 14  
📝 Review:  
This is a well-balanced fun action-adventure game.  I love it!  I wish they would make a James Bond game this fun.  Goldeneye007 for the
Nintendo64 is a bit like this one.  I absolutely cannot get enough of either this game or that one.  If you enjoy spy adventures, or even
just 3d shooters, I would reccomend this.The only negative part is the LOAD screens.  Those really suck.  

🔎 Predicted Sentiment: negative -  Correct Sentiment: POSITIVE

🆔 Review Index: 41266 - Count 15  
📝 Review:  
Baten Kaitos Eternal Wings and the Lost Ocean is an epic game that stretches over 60 hours of role playing gameplay. Besides having an
original universe, story, and characters it is most well known for the unique gameplay with the very innovative and fun Real-Time Card-Based
battle system. Also the graphics and soundtrack on this game is among the best found on the Gamecube or any platform really.The kind of
things that make the real-time card based battle system really fun are:1. Each card has one or more number and you can select the cards
using the numbers to create pairs, strings, etc to create combo attacks that do extra damage2. Each card can evolve and change over time
using the real-time clock so that something like a meat card which is a health item will become rotten meat card over time and then can be
used as an offensive card that causes poisoning3. Different cards when combined logically can yield new cards. A simple example is using a
wood card followed by fire card will yield burnt wood card. Plenty of combinations to discover.4. You the player are a spirit that resides
in the main character. Throughout the game you are prompted with choices. The more your choice is in sync with the game character the more
likely you will be to get defensive cards during the defence and also you will be more likely to get cards during your attack to continue
building your combo.5. Cards can have attributes like water or fire which can cause additional damage (especially if used against enemies
weak to certain element). However if you use a water and fire card in the same attack then you will cancel out the elemental
effect/damage.6. Even though there are large variety of cards they add to variety instead of becoming cumbersome deck management like other
card based games. This one is more about the combos then it is about the card statistics and this keeps the game more action oriented.  

🔎 Predicted Sentiment: positive - Correct Sentiment: POSITIVE

🆔 Review Index: 40468 - Count 16  
📝 Review:  
This is a really fun game. You play a hot and stealthy female ninja seeking revenge on a rival clan. The controls are not bad. The controls
are actually pretty good, especially swinging on the wire. Good control is something I really value in a game, so believe me. This game
would not be possible if it had weak controls (There is alot of acrobatics that you need to perform in order to progress). If you like games
like blood rayne, you should give red ninja a try. This is a much better stealth killer then "manhunt".I do agree that the camera angles can
be a pain sometimes, but that hardly drags the game down to one star. People expect way too much from video games these days. if games are
too easy people whine, and if they are challenging (like Red Ninja) people give up to easy and whine.story=4.5 starssound=3.5
starscontrols=4 starsgraphics=4 starsfun=4.5  

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE

🆔 Review Index: 12195 - Count 17  
📝 Review:  
Resident Evil is the greatest survival horror series ever, no doubt. What you may not know is that the first Resident Evil game was
originally intended as a first person shooter, ala Doom.That didn't happen (thank goodness!) but Capcom DID make a FPS Resident Evil game,
and this is it. This is the first in the Gun Survivor series followed by Resident Evil Dead Aim, Dino Stalker and Code Veronica 2 a Europe
only game that has no bearing on the overall plot.This IS a canon game in the series. The events are mentioned in the opening of Resident
Evil Zero on Gamecube.The graphics are nice because they are from the golden age of playstation and follow in the tradition of Resident Evil
2 and 3. Many complain about this game but it really is not so bad. It's not a shooter as good as Goldeneye for Nintendo 64, of course, but
it's not too shabby in it's own right.The locations are nice and it has that special playstation charm to it, gotta love it. The only reason
this doesn't get 5 stars is because of no light gun support.  

🔎 Predicted Sentiment: negative -  Correct Sentiment: POSITIVE

🆔 Review Index: 11716 - Count 18  
📝 Review:  
This game symbolizes an era gone for me. I was barely a teenager when it released. Never played the first one, but never mind that. This
game is awesome, Squaresoft's answer for Resident Evil. Amazing game. So much fun! The spells and gunplay are great, impressive story and
can frighten at times. Very dark climate. If you never played a PSX game, get yourself a PSX console used and get this game. You will love
it!  

🔎 Predicted Sentiment: negative - Correct Sentiment: POSITIVE

🆔 Review Index: 44540 19 - Count 19  
📝 Review:  
Pariah has been beaten down time and time again, but the funny thing, is that it gets back up every single time. Many people say it sucks
even if they haven't played it but the truth is they have no freaking clue what they are talking about.The graphics are insanely good and
the characters are way better than Halo or Halo2 COMBINED. It is a much better game than both Halo games as well.After playing this I really
think that first person shooters will actually have GOOD stories of wich Halo greatly lacked.Multiplay is a little crazy and I only like to
play it to see what kind of maps people can come up with.I love this game and would choose it over any other fps out there. WAY, WAY better
than Halo 1 or 2. I never want to put the controller down when I play this game.RatingsGraphics:10Sound:10Replayability:8Controls:9Drawn In
Atmosphere:10Fun:10Multiplayer, Co-Op:10, Multiplay:7Overall:9.25Adios, and I'll see your maps on Xbox Live.  

🔎 Predicted Sentiment: negative -  Correct Sentiment: POSITIVE

Let's create a datafram with these examples and for the Ground Truths we just established. We will use these examples as gold examples for the few_shot prompting

In [38]:
grd_truth_data = [
    {"Review": "I have been a huge fan of Harvest Moon since I first played it on my gameboy years ago. Harvest Moon: Back to Nature for PS One has been my favorite, and I'm so glad to know that A Wonderful Life will be bringing back the marriage aspect which I know a lot of people missed with Save The Homeland.  Harvest Moon: A Wonderful Life will bring back my favorite qualities (ie dating, marriage, time progression)with some new features as well. I've heard there will be new farm animals (pigs?) and the ability to cross vegetables (wow!). The animation is as cute as ever, and I can't wait to hear the music. I will be very excited to receive my copy of Harvest Moon: A Wonderful Life.",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "The Game is not even playable single player its online only with 0 Servers. Why do they sell Broken games Wasted time & money for a game i can ever use sad",
     "Zero_sentiment": "positive", "Ground_Truth": "negative"},
    {"Review": "Veery addictive good for those long waits in very boring places (doctors offices and car trips) needs online capability",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "#  Boktai's plot is integrated around its one key gimmick: as Django, you hunt vampires, and you need sunshine (that's right! Real sunshine!) to do it.  Game environments even alter themselves depending on whether it is day or night.  Perfect.  But other aspects of gameplay are remarkable, as well: in keeping with the Metal Gear design, you are urged to sneak around and strategize.  In fact, limited sun-power during nighttime gameplay actually necessitates it.It's true that this a new way of playing.  Still, sometimes the gameplay and puzzles are a little dull and repetitive--which is heartbreakingly disappointing given the attention paid to all other aspects of game design.  Boktai is conceptually thrilling, but the actual game is just sort of... not.  Gameplay simply isn't quite what it should have been.Still, sheer innovation tops Boktai's design flaws.  And absolutely nothing is more exciting than going to the park on a sunny day, finding a bench in a nice patch of light, turning on my GBA, and hearing it say to me, &quot;Today is one great Boktai day!&quot; I can't help but think to myself, &quot;Isn't it?&quot;",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "The superboy plays Gameboy games on your snes (it works on retro duo too!). There is a border for the game that looks like a gameboy. You can also choose borders and color your own, paint and doodle.  For some games (ex: Donkey Kong (arcade) there are special themes borders and...it's in color!  Your  games will sound awesome when played on your t.v., those bleeps and bloops will really shine! You can also change the color pattern of games to your liking.  10/10 highly recommended, buy it!",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "I don't normally play video games, but I'm a huge fan of Civilization 3.  Too huge in fact - it's a real problem for me.  I find myself skipping meals when I play this game.  Civilization 3 is my favorite of the Civilization series.  I think the zone of controls concept is natural to the gameplay, and the ability to have cultural takeovers of other cities is fun.  Some of the oddities and cheapness of the previous versions have been removed - it's not really possible to build every single wonder, for instance.I hadn't played the game in a while, so I decided to get this complete package when I saw it at the store.  However, the several years of updates have really hurt this game.  The problems haven't been addressed - most notably, the diplomacy AI is terrible, and the endgame, with its emphasis on large amounts of indivisual military units, is so ridiculously slow that I can still only rarely make it through without quitting, frustrated.  Instead, the main change is that resources are much more scarce.  These recourses are essential to the progress of your game.  However you have no way of knowing if you'll have, say, a coal resource somewhere in your map, until you've already been playing for 5 hours.If the scarcity was a matter of instigating small military attacks on your neighbors, or heavy diplomacy, that would be fine.  However, this weakness plays off the game's other weakenesses.  Often the necessary resource will only be available halfway around the world.  Launching a military attack is so ridiculously slow that it isn't worth the time even for the most addictive player (that being: me).  And the diplomacy AI is so worthless that other empires won't trade their essential resources even when you throw ridiculous sums of money and technology at them.Anyway, it's no fun to play for hours and hours before you find that your game is a lost cause due to the map you were given.  Also, the expansion disks are (to my mind) worthless.  So I'd strongly recommend just buying the earlier editions of Civilization 3, which are much cheaper.",
     "Zero_sentiment": "positive", "Ground_Truth": "negative"},
    {"Review": "1080 Avalanche is definately a fun game in the sense of speed, but it comes to an overall game, such as tricks and custimization, buy SSX3!Control: 3/5Im sorry, but I have to have big air and tricks. The way I see it, if I can't pull of awesome combos, things are bad. But, the system where you can get back on balance if you mess up a jump is cool.Sound: 5/5The reason Sound gets 5? I LOVE THE SOUNDTRACK. The song &quot;Choke&quot; is amazing, and is really something that you could snowboard to. The sound of the snow under the board is crisp as well.Modes and Extras: 2/5The Match Race is pretty much the only thing worth it, and even that isn't very good. The characters are limited, the boards are limited, and there isn't really any customization or unlockables.Overall: 3/5The game is cool if you like speed, but the tricks suck and you can only hold a jump for 2 seconds. I reccomend you buy SSX3 instead of this, or you can buy this too. Whatever floats your boat!",
     "Zero_sentiment": "positive", "Ground_Truth": "negative"},
    {"Review": "The story was OK, I was not taken away with it since it's the usual 'Last of species, princess in trouble, hero with daddy-problem' stuff.The characters had some personality, it was never developed beyond looking like comic characters, I just never took them seriously.The graphics are ugly, I do not like that they are so small, I could barely see Vivi.But the absolute worst part is the battle-system, it almost takes a whole minute just to load all the characters, monsters and angle the camera correctly.So much time is wasted just wating for the battle to load.So when you meet weak monsters you kill with a blow, you seriously get more than annoyed because it takes longer time to load the fight than to actually finish it!Sad way to ruin a possibly could be fantastic game.",
     "Zero_sentiment": "positive", "Ground_Truth": "negative"},
    {"Review": "When you add the two together you should come up with one heck of a game. What happened here is that the strategic, political and conscript aspects from ROTK were summarized into pairs, and issued to you as suggestions by your leaders.  You can refuse their ideas, and save your gold, which you need to hire captured enemy generals.  The map is a bit smaller than ROTK and with far less provinces.  Despite this, the game is incredible.  If you have ever wanted to fight a la DW while you were playing ROTK, this is it...  You make your plans, hire men, whatever, then choose to attack or perhaps you sit and watch this turn unfold, saving money and manpower.  Unlike DW though, the gates are everywhere, and they are now strongholds, which you must take to gain the edge.  Also, generals can die 3 times in one battle.  Oh yeah, they come back 3 times, and get harder each time.  In a normal battle you might have to kill 15 generals. Actually only 5 but you did it 3 times each.  They re-appear out of nowhere with a portion of their original force.  Kind of annoying.  A cool aspect of this is... For once you can finally be the only general on the screen, if you want that is.  Send yourself in alone against 50000 enemy troops.  The weapons and items are the same but harder to come by.  Your provinces give you certain items, that you can only use once every 5 turns.  Be carefull what you use.  Your weapon gets stronger based on your level, just like DW.  The number of men under your command is based on your achievements like in ROTK.  Form alliances and help defend other lords.  You gain rewards for this.  During battle your AI generals can use their 'skills' once per battle like in ROTK.  Create an officer is better but nothing to get excited about.  You can choose from a few different looks but the bottom line is you still look stupid.  Oh, and I have yet to see any of the trademark Bodyguards.  I miss that. Basically, Koei has taken good parts of each game and put it together.  What more could you want?  Oh yeah, no two player co-op, so I guess that is one thing you could want.",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "This game could have been a heck of alot better. The idea of the game is pretty stupid. They should have taken things from different episodes and made that the game. The graphics are good, since it's only playstation. The multiplayer is pretty fun. Too bad this game wasn't released later on and they could have put Jimmy and Timmy in it. I'm glad I bought this game though.",
     "Zero_sentiment": "positive", "Ground_Truth": "positive"},
    {"Review": "This is a great game. As another reviewer said, they could have sold a million copies of a mediocre game, but this one is very good. My seven year old daughter and I played 'Socerer's Stone' first. We enjoyed it, but it was a bit clunky. Next we started playing this one. We're almost 20 hours into it already, less than half way through, and having a great time. This one has great replay value-we're going to go back and find the wizard cards we missed.This one took everything that was good about the first game, got rid of what didn't work, and added a lot more. So you get more spells, more places to explore, more options, more fun.If you get this in the 'World of Harry Potter' collection, you'll get four games for twenty smackers. Great value!",
     "Zero_sentiment": "positive", "Ground_Truth": "positive"},
    {"Review": "This is a game worth playing and since it's been out about 2 years or a little longer it's affordable. I'm a fan of the original Wolfenstein 3D and this game is still worthy of the name. Great game play, sound, weapons and decent graphics. Another plus about this is when you beat the game you can play the original Wolfenstein 3D game, this is definally worth buying. Once you get used to the controls and how the game is played out you'll get hooked.",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "good, but games like golden sun are a lot better. heres how it lays out:graphics:2sound:3 nothing specialstory:4gameplay:3 too much magicoverall:3.5 if u want an Rpg, get golden sun instead.",
    "Zero_sentiment": "positive", "Ground_Truth": "positive"},
    {"Review": "This is a well-balanced fun action-adventure game.  I love it!  I wish they would make a James Bond game this fun.  Goldeneye007 for the Nintendo64 is a bit like this one.  I absolutely cannot get enough of either this game or that one.  If you enjoy spy adventures, or even just 3d shooters, I would reccomend this.The only negative part is the LOAD screens.  Those really suck.",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "Baten Kaitos Eternal Wings and the Lost Ocean is an epic game that stretches over 60 hours of role playing gameplay. Besides having an original universe, story, and characters it is most well known for the unique gameplay with the very innovative and fun Real-Time Card-Based battle system. Also the graphics and soundtrack on this game is among the best found on the Gamecube or any platform really.The kind of things that make the real-time card based battle system really fun are:1. Each card has one or more number and you can select the cards using the numbers to create pairs, strings, etc to create combo attacks that do extra damage2. Each card can evolve and change over time using the real-time clock so that something like a meat card which is a health item will become rotten meat card over time and then can be used as an offensive card that causes poisoning3. Different cards when combined logically can yield new cards. A simple example is using a wood card followed by fire card will yield burnt wood card. Plenty of combinations to discover.4. You the player are a spirit that resides in the main character. Throughout the game you are prompted with choices. The more your choice is in sync with the game character the more likely you will be to get defensive cards during the defence and also you will be more likely to get cards during your attack to continue building your combo.5. Cards can have attributes like water or fire which can cause additional damage (especially if used against enemies weak to certain element). However if you use a water and fire card in the same attack then you will cancel out the elemental effect/damage.6. Even though there are large variety of cards they add to variety instead of becoming cumbersome deck management like other card based games. This one is more about the combos then it is about the card statistics and this keeps the game more action oriented.",
     "Zero_sentiment": "positive", "Ground_Truth": "positive"},
    {"Review": "This is a really fun game. You play a hot and stealthy female ninja seeking revenge on a rival clan. The controls are not bad. The controls are actually pretty good, especially swinging on the wire. Good control is something I really value in a game, so believe me. This game would not be possible if it had weak controls (There is alot of acrobatics that you need to perform in order to progress). If you like games like blood rayne, you should give red ninja a try. This is a much better stealth killer then 'manhunt'.I do agree that the camera angles can be a pain sometimes, but that hardly drags the game down to one star. People expect way too much from video games these days. if games are too easy people whine, and if they are challenging (like Red Ninja) people give up to easy and whine.story=4.5 starssound=3.5 starscontrols=4 starsgraphics=4 starsfun=4.5",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "Resident Evil is the greatest survival horror series ever, no doubt. What you may not know is that the first Resident Evil game was originally intended as a first person shooter, ala Doom.That didn't happen (thank goodness!) but Capcom DID make a FPS Resident Evil game, and this is it. This is the first in the Gun Survivor series followed by Resident Evil Dead Aim, Dino Stalker and Code Veronica 2 a Europe only game that has no bearing on the overall plot.This IS a canon game in the series. The events are mentioned in the opening of Resident Evil Zero on Gamecube.The graphics are nice because they are from the golden age of playstation and follow in the tradition of Resident Evil 2 and 3. Many complain about this game but it really is not so bad. It's not a shooter as good as Goldeneye for Nintendo 64, of course, but it's not too shabby in it's own right.The locations are nice and it has that special playstation charm to it, gotta love it. The only reason this doesn't get 5 stars is because of no light gun support.",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": "This game symbolizes an era gone for me. I was barely a teenager when it released. Never played the first one, but never mind that. This game is awesome, Squaresoft's answer for Resident Evil. Amazing game. So much fun! The spells and gunplay are great, impressive story and can frighten at times. Very dark climate. If you never played a PSX game, get yourself a PSX console used and get this game. You will love it!",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
    {"Review": " Pariah has been beaten down time and time again, but the funny thing, is that it gets back up every single time. Many people say it sucks even if they haven't played it but the truth is they have no freaking clue what they are talking about.The graphics are insanely good and the characters are way better than Halo or Halo2 COMBINED. It is a much better game than both Halo games as well.After playing this I really think that first person shooters will actually have GOOD stories of wich Halo greatly lacked.Multiplay is a little crazy and I only like to play it to see what kind of maps people can come up with.I love this game and would choose it over any other fps out there. WAY, WAY better than Halo 1 or 2. I never want to put the controller down when I play this game.RatingsGraphics:10Sound:10Replayability:8Controls:9Drawn In Atmosphere:10Fun:10Multiplayer, Co-Op:10, Multiplay:7Overall:9.25Adios, and I'll see your maps on Xbox Live.",
     "Zero_sentiment": "negative", "Ground_Truth": "positive"},
]

In [39]:
# Creating dataframe where the evaluation of the classification is performed

df_gt = pd.DataFrame(grd_truth_data)

In [40]:
# Visualizing said df

df_gt

,Review,Zero_sentiment,Ground_Truth
0,"I have been a huge fan of Harvest Moon since I first played it on my gameboy years ago. Harvest Moon: Back to Nature for PS One has been my favorite, and I'm so glad to know that A Wonderful Life will be bringing back the marriage aspect which I know a lot of people missed with Save The Homeland. Harvest Moon: A Wonderful Life will bring back my favorite qualities (ie dating, marriage, time progression)with some new features as well. I've heard there will be new farm animals (pigs?) and the ability to cross vegetables (wow!). The animation is as cute as ever, and I can't wait to hear the music. I will be very excited to receive my copy of Harvest Moon: A Wonderful Life.",negative,positive
1,The Game is not even playable single player its online only with 0 Servers. Why do they sell Broken games Wasted time & money for a game i can ever use sad,positive,negative
2,Veery addictive good for those long waits in very boring places (doctors offices and car trips) needs online capability,negative,positive
3,"# Boktai's plot is integrated around its one key gimmick: as Django, you hunt vampires, and you need sunshine (that's right! Real sunshine!) to do it. Game environments even alter themselves depending on whether it is day or night. Perfect. But other aspects of gameplay are remarkable, as well: in keeping with the Metal Gear design, you are urged to sneak around and strategize. In fact, limited sun-power during nighttime gameplay actually necessitates it.It's true that this a new way of playing. Still, sometimes the gameplay and puzzles are a little dull and repetitive--which is heartbreakingly disappointing given the attention paid to all other aspects of game design. Boktai is conceptually thrilling, but the actual game is just sort of... not. Gameplay simply isn't quite what it should have been.Still, sheer innovation tops Boktai's design flaws. And absolutely nothing is more exciting than going to the park on a sunny day, finding a bench in a nice patch of light, turning on my GBA, and hearing it say to me, &quot;Today is one great Boktai day!&quot; I can't help but think to myself, &quot;Isn't it?&quot;",negative,positive
4,"The superboy plays Gameboy games on your snes (it works on retro duo too!). There is a border for the game that looks like a gameboy. You can also choose borders and color your own, paint and doodle. For some games (ex: Donkey Kong (arcade) there are special themes borders and...it's in color! Your games will sound awesome when played on your t.v., those bleeps and bloops will really shine! You can also change the color pattern of games to your liking. 10/10 highly recommended, buy it!",negative,positive
5,"I don't normally play video games, but I'm a huge fan of Civilization 3. Too huge in fact - it's a real problem for me. I find myself skipping meals when I play this game. Civilization 3 is my favorite of the Civilization series. I think the zone of controls concept is natural to the gameplay, and the ability to have cultural takeovers of other cities is fun. Some of the oddities and cheapness of the previous versions have been removed - it's not really possible to build every single wonder, for instance.I hadn't played the game in a while, so I decided to get this complete package when I saw it at the store. However, the several years of updates have really hurt this game. The problems haven't been addressed - most notably, the diplomacy AI is terrible, and the endgame, with its emphasis on large amounts of indivisual military units, is so ridiculously slow that I can still only rarely make it through without quitting, frustrated. Instead, the main change is that resources are much more scarce. These recourses are essential to the progress of your game. However you have no way of knowing if you'll have, say, a coal resource somewhere in your map, until you've already been playing for 5 hours.If the scarcity was a matter of instigating small military attacks on y

Crafting Gold Examples for the few_shot prompt

In [41]:
# Defining variable gold_examples with columns from df_gt

gold_examples = df_gt[["Review", "Ground_Truth"]].to_json(orient='records')

In [42]:
# Converting examples to Json format for future use

json.loads(gold_examples)

[{'Review': "I have been a huge fan of Harvest Moon since I first played it on my gameboy years ago. Harvest Moon: Back to Nature for PS One has been my favorite, and I'm so glad to know that A Wonderful Life will be bringing back the marriage aspect which I know a lot of people missed with Save The Homeland.  Harvest Moon: A Wonderful Life will bring back my favorite qualities (ie dating, marriage, time progression)with some new features as well. I've heard there will be new farm animals (pigs?) and the ability to cross vegetables (wow!). The animation is as cute as ever, and I can't wait to hear the music. I will be very excited to receive my copy of Harvest Moon: A Wonderful Life.",
  'Ground_Truth': 'positive'},
 {'Review': 'The Game is not even playable single player its online only with 0 Servers. Why do they sell Broken games Wasted time & money for a game i can ever use sad',
  'Ground_Truth': 'negative'},
 {'Review': 'Veery addictive good for those long waits in very boring pl

In [43]:
# User message for prompt

user_message_template = "```{reviewText}```"


In [44]:
# system message for prompt

few_shot_system_message = """
Classify reviews in the input as positive or negative in sentiment.
Reviews will be delimited by triple backticks, that is, ```.
Do not explain your answer. Your answer should only contain the label: positive or negative.
"""

In [45]:
# Defining function to create few_shot_prompt

def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for customer reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['Review']
        example_sentiment = example['Ground_Truth']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content':  user_message_template.format(
                    reviewText=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant',
             'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

In [46]:
# Creating Few shot prompt

few_shot_prompt = create_prompt(
    few_shot_system_message,
    gold_examples,
    user_message_template
)

In [47]:
# Verifying few_shot prompt content

few_shot_prompt

[{'role': 'system',
  'content': '\nClassify reviews in the input as positive or negative in sentiment.\nReviews will be delimited by triple backticks, that is, ```.\nDo not explain your answer. Your answer should only contain the label: positive or negative.\n'},
 {'role': 'user',
  'content': "```I have been a huge fan of Harvest Moon since I first played it on my gameboy years ago. Harvest Moon: Back to Nature for PS One has been my favorite, and I'm so glad to know that A Wonderful Life will be bringing back the marriage aspect which I know a lot of people missed with Save The Homeland.  Harvest Moon: A Wonderful Life will bring back my favorite qualities (ie dating, marriage, time progression)with some new features as well. I've heard there will be new farm animals (pigs?) and the ability to cross vegetables (wow!). The animation is as cute as ever, and I can't wait to hear the music. I will be very excited to receive my copy of Harvest Moon: A Wonderful Life.```"},
 {'role': 'ass

In [48]:
# Getting the token count for few_shot_prompt

num_tokens_from_messages(few_shot_prompt)

3865

In [51]:
# testing classification prompt with a sample

import textwrap

# Step 1: Build the few-shot prompt using gold examples (already in JSON format)
few_shot_prompt = create_prompt(
    system_message=few_shot_system_message,
    examples=gold_examples,
    user_message_template="```{reviewText}```"
)

# Step 2: Classify 5 new random reviews from df['reviewText']
sample_indices = random.sample(range(len(df)), 10)

     # initiating the loop
for count, idx in enumerate(sample_indices, start=1):
    review_text = df['reviewText'].iloc[idx]

    # Build full prompt: few-shot + one new user review
    full_prompt = few_shot_prompt + [
        {
            'role': 'user',
            'content': user_message_template.format(reviewText=review_text)
        }
    ]

    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=full_prompt,
            temperature=0,
            max_tokens=4
        )

        sentiment = response.choices[0].message.content.strip()

        # Display output
        print(f"\n🔢 Review #{count} (Index: {idx})")
        print("📝 Review:\n", textwrap.fill(review_text, width=120))
        print("🔎 Predicted Sentiment (Few-Shot):", sentiment)
        print("=" * 100)

    except Exception as e:
        print(f"\n❌ Error on review #{count} (Index: {idx}): {e}")



🔢 Review #1 (Index: 22127)
📝 Review:
 Rockstar was the king of the moment with their huge success with Grand Theft Auto 3 for the Playstation 2. People were
hoping for the same quality game when they heard about State of Emergency...Unfortunatly, the quality was alot
less...Instead of a realistic cartoon world of violence, State of Emergency gave people a horrible anime cheap arcade
game. To compare this game to Grand Theft Auto is to compare need for speed with crazy taxi.Sure a game like Crazy Taxi
is fun, but after 5 minutes you realize how cheap the game is and how repetative it becomes. The same is true for State
of Emergency.Graphics=2Sound and music=2Control=3.75Fun Factor=3.5Replay value=3Total=2.85You might not regret owning it
for about 5 dollars. I feel bad for the people who paid full price.Don't they test and play games before they sell
them??? geez!
🔎 Predicted Sentiment (Few-Shot): negative

🔢 Review #2 (Index: 39606)
📝 Review:
 Although I was not a huge fan of the orig

## **6. Scaling the classification**

The 10 samples are correctly classified. Time to scale it!

In [52]:
# Checking the length of the data

len(data_100min)

5674

In [53]:
# To check progress

!pip install tqdm

In [57]:
# Running the data in batches of 250 each. Storing in csv file.

import time
from tqdm import tqdm
import os

# --- Parameters ---
BATCH_SIZE = 250
OUTPUT_FILE = "few_shot_sentiment_predictions.csv"

# --- Initialize prompt ---
few_shot_prompt = create_prompt(
    system_message=few_shot_system_message,
    examples=gold_examples,
    user_message_template="```{reviewText}```"
)

# --- Load existing predictions (if any) ---
if os.path.exists(OUTPUT_FILE):
    df_predictions = pd.read_csv(OUTPUT_FILE)
    start_idx = df_predictions["Index"].max() + 1
    print(f"Resuming from index {start_idx}...")
else:
    df_predictions = pd.DataFrame(columns=["Review_Index", "asin", "Review", "Prediction"])
    start_idx = 0

# --- Run in batches ---
reviews = data_100min["reviewText"]
total_reviews = len(reviews)

for batch_start in range(start_idx, total_reviews, BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, total_reviews)
    batch_predictions = []

    print(f"\n🔁 Processing batch {batch_start} to {batch_end - 1}")

    for i in tqdm(range(batch_start, batch_end), desc="Classifying reviews"):
      review_text = reviews.iloc[i]
      asin = data_100min["asin"].iloc[i]  # Track the product ID

    full_prompt = few_shot_prompt + [
        {
            'role': 'user',
            'content': user_message_template.format(reviewText=review_text)
        }
    ]

    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=full_prompt,
            temperature=0,
            max_tokens=4
        )
        sentiment = response.choices[0].message.content.strip()

    except Exception as e:
        sentiment = f"ERROR: {str(e)}"

    batch_predictions.append({
        "Index": i,
        "ASIN": asin,
        "Review": review_text,
        "Prediction": sentiment
    })

    time.sleep(0.5)  # Adjust this as needed

    # --- Append batch to main DataFrame ---
    df_predictions = pd.concat([df_predictions, pd.DataFrame(batch_predictions)], ignore_index=True)

    # --- Save progress ---
    df_predictions.to_csv(OUTPUT_FILE, index=False)
    print(f"✅ Batch saved: {OUTPUT_FILE}")

print("\n🎉 All batches processed!")



🔁 Processing batch 0 to 249


Classifying reviews: 100%|██████████| 250/250 [00:00<00:00, 53654.81it/s]


✅ Batch saved: few_shot_sentiment_predictions.csv

🔁 Processing batch 250 to 499


Classifying reviews: 100%|██████████| 250/250 [00:00<00:00, 57702.84it/s]


✅ Batch saved: few_shot_sentiment_predictions.csv

🔁 Processing batch 500 to 749


...

✅ Batch saved: few_shot_sentiment_predictions.csv

🔁 Processing batch 5250 to 5499


Classifying reviews: 100%|██████████| 250/250 [00:00<00:00, 54621.87it/s]


✅ Batch saved: few_shot_sentiment_predictions.csv

🔁 Processing batch 5500 to 5673


Classifying reviews: 100%|██████████| 174/174 [00:00<00:00, 48683.14it/s]


✅ Batch saved: few_shot_sentiment_predictions.csv

🎉 All batches processed!


## **7. Ranking the results**

Now comes the fun part. I am starting the process to compile the 10 best and 10 worst ranked games as per the reviews of Amazon customers for the period form October 14, 1999 to JUly 22, 2014:

In [58]:
summary = df_predictions.groupby("ASIN")["Prediction"].value_counts().unstack(fill_value=0)
summary = summary.rename(columns={"positive": "Positive_Reviews", "negative": "Negative_Reviews"})
summary["Total_Reviews"] = summary["Positive_Reviews"] + summary["Negative_Reviews"]
summary = summary.reset_index()


I will introduce the ratings given to calculate a metric for the games

In [59]:
# Merge ratings from original data
ratings = data_100min.groupby("asin")["overall"].mean().reset_index()
ratings = ratings.rename(columns={"asin": "ASIN", "overall": "Avg_Rating"})

# Merge with sentiment summary
summary = pd.merge(summary, ratings, on="ASIN", how="left")

# Sentiment ratio (positive / total)
summary["Sentiment_Ratio"] = summary["Positive_Reviews"] / summary["Total_Reviews"]

# Final Score
summary["Final_Score"] = summary["Sentiment_Ratio"] + summary["Avg_Rating"]


Let's obtain the top 10 and 10 worst ranked, as per defined metric

In [60]:
summary_sorted = summary.sort_values("Final_Score", ascending=False)
top_10 = summary_sorted.head(10).reset_index(drop=True)
bottom_10 = summary_sorted.tail(10).sort_values("Final_Score",ascending=True).reset_index(drop=True)


In [61]:
top_10

,ASIN,Negative_Reviews,Positive_Reviews,Total_Reviews,Avg_Rating,Sentiment_Ratio,Final_Score
0,B00005Q8M0,0,1,1,3.742718,1.0,4.742718
1,B00000JRSB,0,1,1,3.677083,1.0,4.677083
2,B00004YRQA,0,1,1,3.662338,1.0,4.662338
3,B00005O0I2,0,1,1,3.627358,1.0,4.627358
4,B00005QEFF,0,1,1,3.623932,1.0,4.623932
5,B0001VGFK2,0,1,1,3.582192,1.0,4.582192
6,B00004Y57G,0,1,1,3.542857,1.0,4.542857
7,B00005NZ1G,0,1,1,3.538961,1.0,4.538961
8,B0000696CZ,0,1,1,3.517241,1.0,4.517241
9,B000067DPM,0,1,1,3.504587,1.0,4.504587


In [62]:
bottom_10

,ASIN,Negative_Reviews,Positive_Reviews,Total_Reviews,Avg_Rating,Sentiment_Ratio,Final_Score
0,B0007TFLLC,1,0,1,3.097087,0.0,3.097087
1,B0000296O5,0,1,1,3.168675,1.0,4.168675
2,B00005ML10,0,1,1,3.220588,1.0,4.220588
3,B000066JRN,0,1,1,3.231343,1.0,4.231343
4,B00008J7NZ,0,1,1,3.285000,1.0,4.285000
5,B00005TNI6,0,1,1,3.294118,1.0,4.294118
6,B0009VXAM0,0,1,1,3.319149,1.0,4.319149
7,B000084318,0,1,1,3.339869,1.0,4.339869
8,B00064MUIA,0,1,1,3.370370,1.0,4.370370
9,B00006FWTX,0,1,1,3.398058,1.0,4.398058


Since the dataset lacks the video game names, we will use an API to retrieve them from publicly available sources.

For this task, we use the Tavily Search API to perform real-time web lookups.

In [64]:
# Installin Tavily

!pip install tavily-python

In [87]:
# Authenticating Tavily credentials

import json

# Load the key from the JSON file
with open("tavily_key.json", "r") as f:
    data = f.read()

# Parse the JSON data
api_data = json.loads(data)

api_key = api_data["TAVILY_API_KEY"]

In [89]:
# Code to retrieve names - testing

from tavily import TavilyClient

# Initialize the client
tavily_client = TavilyClient(api_key=api_key)

# Example usage
response = tavily_client.search("Amazon video game ASIN B000084318")
print(response)


{'query': 'Amazon video game ASIN B000084318', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.amazon.com/Legend-Zelda-Wind-Waker-Gamecube/dp/B000084318', 'title': 'The Legend of Zelda: The Wind Waker - Amazon.com', 'content': 'The Legend of Zelda: The Wind Waker (Renewed)Amazon Renewed  4.8 out of 5 stars 21  GameCube  -12%$103.88$103.88 Typical price:$118.58   Get it as soon as Wednesday, May 28  FREE Shipping by Amazon Only 1 left in stock - order soon. The Legend of Zelda: Ocarina of Time (w/ Master Quest) (Renewed)Amazon Renewed  3.8 out of 5 stars 22  GameCube  -18%$74.18$74.18 List Price:$89.99   Get it May 29 - Jun 2  FREE Shipping Only 2 left in stock - order soon. Legend of Zelda The Wind Waker - Gamecube (Renewed)Amazon Renewed  4.3 out of 5 stars 72 -26%$95.57$95.57 List Price:$129.00   Get it May 29 - Jun 2  FREE Shipping Only 2 left in stock - order soon.', 'score': 0.21075045, 'raw_content': None}, {'url': 'https://www.amazon.c

In [90]:
# Additional confirmation test

response = tavily_client.search("Amazon video game ASIN B00005O0I2")
print(response)

{'query': 'Amazon video game ASIN B00005O0I2', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.amazon.sg/Rockstar-Games-Grand-Theft-Auto/dp/B00005O0I2', 'title': 'Grand Theft Auto 3 : Unknown: Amazon.sg: Video Games', 'content': 'Manufacturer: UnAssigned; ASIN: B00005O0I2; Manufacturer reference: 27079; Average Customer Review : 4.3 4.3 out of 5 stars (1,119). Customer reviews.', 'score': 0.8812988, 'raw_content': None}, {'url': 'https://www.amazon.com/Grand-Theft-Auto-III-Pc/dp/B00005O0I2', 'title': 'Rockstar Games Grand Theft Auto 3 : Unknown - Amazon.com', 'content': '* Games This item:  Rockstar Games Grand Theft Auto 3 "Grand Theft Auto III is a **great game**, filled with action & cars, cars, cars. "**Great game**" Read more Customers report that the game works well. Some of the best free-world experience/movement in any game...." Read more Purchased via Amazon and game was delivered a few days earlier than promised...." Read more "Gran

The test to retrive the product information for two asin worked. Now I'm scaling and getting those titles for both lists

In [94]:
# Running the query

def get_title_from_asin(asin):
    query = f"What is the title of the Amazon video game with ASIN {asin}?"
    try:
        response = tavily_client.search(query, search_depth="basic")
        return response['results'][0]['title'] if response['results'] else "Not found"
    except Exception as e:
        return f"Error: {str(e)}"

Now, let's add a column Game_name to each dataframe: top_10 and bottom_10

In [95]:
# Apply to top_10 and bottom_10
top_10['Game_name'] = top_10['ASIN'].apply(lambda asin: get_title_from_asin(asin))
time.sleep(1)  # Small pause between batches if needed
bottom_10['Game_name'] = bottom_10['ASIN'].apply(lambda asin: get_title_from_asin(asin))

In [96]:
top_10

,ASIN,Negative_Reviews,Positive_Reviews,Total_Reviews,Avg_Rating,Sentiment_Ratio,Final_Score,Game_name
0,B00005Q8M0,0,1,1,3.742718,1.0,4.742718,Super Smash Bros Melee : Gamecube: Video Games - Amazon.com
1,B00000JRSB,0,1,1,3.677083,1.0,4.677083,Final Fantasy VII - PlayStation : Amazon.sg: Video Games
2,B00004YRQA,0,1,1,3.662338,1.0,4.662338,Amazon Games - Wikipedia
3,B00005O0I2,0,1,1,3.627358,1.0,4.627358,Grand Theft Auto III - Playstation 2 : Amazon.ca: Video Games
4,B00005QEFF,0,1,1,3.623932,1.0,4.623932,Amazon.com: GameCube Console - Indigo
5,B0001VGFK2,0,1,1,3.582192,1.0,4.582192,[DEAD] GTA San Andreas for $29.99 (CDN) at Amazon.ca
6,B00004Y57G,0,1,1,3.542857,1.0,4.542857,Final Fantasy IX : Playstation: Video Games - Amazon.com
7,B00005NZ1G,0,1,1,3.538961,1.0,4.538961,Halo/Game - Amazon.se
8,B0000696CZ,0,1,1,3.517241,1.0,4.517241,Grand Theft Auto: Vice City / Game - Amazon UK
9,B000067DPM,0,1,1,3.504587,1.0,4.504587,Amazon.com: Star Wars Knights of the Old Republic - Xbox


In [98]:
bottom_10

,ASIN,Negative_Reviews,Positive_Reviews,Total_Reviews,Avg_Rating,Sentiment_Ratio,Final_Score,Game_name
0,B0007TFLLC,1,0,1,3.097087,0.0,3.097087,PlayStation Portable (PSP) Value Pack - Amazon.com
1,B0000296O5,0,1,1,3.168675,1.0,4.168675,Final Fantasy VIII : Playstation: Amazon.in: Video Games
2,B00005ML10,0,1,1,3.220588,1.0,4.220588,Metal Gear Solid 2: Sons of Liberty - Amazon.sg
3,B000066JRN,0,1,1,3.231343,1.0,4.231343,Ultimate Nex Playground Console Review - Guilt Free Screen Time
4,B00008J7NZ,0,1,1,3.285000,1.0,4.285000,Halo 2 - Xbox : microsoft: Video Games - Amazon.com
5,B00005TNI6,0,1,1,3.294118,1.0,4.294118,Final Fantasy X : Square Enix LLC: Video Games - Amazon.com
6,B0009VXAM0,0,1,1,3.319149,1.0,4.319149,Playstation 3 60GB System - Amazon.se
7,B000084318,0,1,1,3.339869,1.0,4.339869,I got the game Amazon Trail in a box of cereal as a kid. I spent so ...
8,B00064MUIA,0,1,1,3.370370,1.0,4.370370,Amazon Games - Wikipedia
9,B00006FWTX,0,1,1,3.398058,1.0,4.398058,Animal Crossing : Unknown: Video Games - Amazon.com


***NOTE***: Not all data included in this dataset refers to video games. It also includes ratings for game consoles and others.  
**For top_10:**  
B00004YRQA: PlayStation 2 8MB Memory Card  
B00005QEFF - It's a console as detailed on table  
**For bottom_10:**  
B00064MUIA - Nintendo DS Bundle with Metroid Prime Hunters Demo  
B000084318 - Refers to offers from Amazon for Gift cards, Consoles, and games, including Zelda  
B0009VXAM0 - Console PlayStation 3  
B00005ML10 - refers to cases to protect playground game systems  
B0007TFLLC - it's the PlayStation Portable (PSP) Value Pack

## 8. **Presenting the results**

In [101]:
# Ranking tabultation

from IPython.display import display, HTML

# Create section headers as styled HTML rows (spanning all 4 columns)
top_label = pd.DataFrame(
    [["", "<b><span style='font-size:14pt'>TOP 10 RANKED VIDEO GAMES & OTHERS</span></b>", "", ""]],
    columns=["Rank", "ASIN", "Game_name", "Final_Score"]
)

bottom_label = pd.DataFrame(
    [["", "<b><span style='font-size:14pt'>WORST 10 RANKED VIDEO GAMES & OTHERS</span></b>", "", ""]],
    columns=["Rank", "ASIN", "Game_name", "Final_Score"]
)

# Reset index for clean Index values
top_summary = top_10[["ASIN", "Game_name", "Final_Score"]].reset_index(drop=True)
top_summary.insert(0, "Rank", range(1, 11))

bottom_summary = bottom_10[["ASIN", "Game_name", "Final_Score"]].reset_index(drop=True)
bottom_summary.insert(0, "Rank", range(1, 11))

# Combine everything
summary_df = pd.concat([
    top_label,
    top_summary,
    bottom_label,
    bottom_summary
], ignore_index=True)

# Display using HTML (so bold/centered headers render properly)
display(HTML(summary_df.to_html(escape=False, index=False)))


Rank,ASIN,Game_name,Final_Score
,TOP 10 RANKED VIDEO GAMES & OTHERS,,
1,B00005Q8M0,Super Smash Bros Melee : Gamecube: Video Games - Amazon.com,4.742718
2,B00000JRSB,Final Fantasy VII - PlayStation : Amazon.sg: Video Games,4.677083
3,B00004YRQA,Amazon Games - Wikipedia,4.662338
4,B00005O0I2,Grand Theft Auto III - Playstation 2 : Amazon.ca: Video Games,4.627358
5,B00005QEFF,Amazon.com: GameCube Console - Indigo,4.623932
6,B0001VGFK2,[DEAD] GTA San Andreas for $29.99 (CDN) at Amazon.ca,4.582192
7,B00004Y57G,Final Fantasy IX : Playstation: Video Games - Amazon.com,4.542857
8,B00005NZ1G,Halo/Game - Amazon.se,4.538961
9,B0000696CZ,Grand Theft Auto: Vice City / Game - Amazon UK,4.517241


## **Conclusion**

In this project, we successfully built a robust sentiment classification system for Amazon video game reviews using a **few-shot prompting approach with a large language model (LLM)**. The process included filtering and processing thousands of user reviews, classifying them as either **positive** or **negative**, and aggregating the results at the product (ASIN) level.

To enrich our insights, we retrieved game titles using the Tavily search API, cross-referenced with ASINs, and compiled a final ranking of the **top 10 and bottom 10 video games** based on the ratio of positive to total reviews.

Key accomplishments include:

* ✅ Implemented a few-shot prompting strategy with example-based guidance.
* ✅ Processed and classified thousands of reviews in efficient batches with intermediate saving for fault tolerance.
* ✅ Merged predictions into a clean output format for later reuse.
* ✅ Retrieved and matched game titles using a search API for clearer presentation.
* ✅ Computed and displayed final scores per product, ranking games based on sentiment outcomes.
* ✅ Cleanly visualized the top and bottom performers using styled HTML for presentation.

This end-to-end pipeline is fully **reproducible and scalable**, meaning it can be reused to evaluate new or unseen datasets with minimal changes. By simply replacing the input dataset, the same code structure can be applied to any collection of product reviews, enabling consistent and reliable sentiment-based product ranking.
